<a href="https://colab.research.google.com/github/yanter5/march_machine_learning/blob/main/march_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
games_s = pd.read_csv("gdrive/My Drive/march_madness/MRegularSeasonCompactResults.csv")
games_l = pd.read_csv("gdrive/My Drive/march_madness/MRegularSeasonDetailedResults.csv")
tourney_s = pd.read_csv("gdrive/My Drive/march_madness/MNCAATourneyCompactResults.csv")
tourney_l = pd.read_csv("gdrive/My Drive/march_madness/MNCAATourneyDetailedResults.csv")
massey = pd.read_csv("gdrive/My Drive/march_madness/MMasseyOrdinals.csv")

In [ ]:
df_games = games_l.loc[games_l['Season'] >= 2003]
df_games  #Pull all games played during or after the 2003 season

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100418,2022,98,1400,79,1242,76,H,0,28,67,...,13,15,23,5,24,10,15,3,5,21
100419,2022,98,1411,66,1126,63,A,0,24,59,...,21,15,24,5,23,10,19,13,2,23
100420,2022,98,1422,68,1441,49,A,0,23,56,...,24,8,11,10,18,5,16,8,2,12
100421,2022,98,1438,69,1181,68,A,0,31,65,...,17,18,22,11,25,14,14,3,9,11


In [ ]:
massey.loc[massey['Season'] == 2003]  #Massey contains team rankings of each team for every week of each season

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260
...,...,...,...,...,...
131530,2003,133,WTE,1460,218
131531,2003,133,WTE,1461,70
131532,2003,133,WTE,1462,12
131533,2003,133,WTE,1463,107


In [ ]:
teamstats_db = {} #populate dictionary with end of season rankings, as that would intuitively be the most accurate reading of team strength for the team each season
for i in range(2003, 2023): #I took the mean rank for each ranking stat in the massey database (end of season/most recent for 2022)
  if i == 2022:
    masseySub = massey.loc[(massey['Season'] == i) & (massey['RankingDayNum'] == 100)]
  else:
    masseySub = massey.loc[(massey['Season'] == i) & (massey['RankingDayNum'] == 133)]
  ids = []
  ranks=[]
  for id in masseySub['TeamID'].unique():
    teamRank = masseySub.loc[massey['TeamID'] == id]['OrdinalRank'].mean()
    ids.append(id)
    ranks.append(teamRank)
  year_ranks = pd.DataFrame()
  year_ranks['TeamID'] = ids
  year_ranks['rank'] = ranks
  teamstats_db[i] = year_ranks

In [ ]:
masseySub

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
4502314,2022,100,7OT,1101,129
4502315,2022,100,7OT,1102,252
4502316,2022,100,7OT,1103,156
4502317,2022,100,7OT,1104,13
4502318,2022,100,7OT,1105,352
...,...,...,...,...,...
4521715,2022,100,WOL,1468,183
4521716,2022,100,WOL,1469,259
4521717,2022,100,WOL,1470,209
4521718,2022,100,WOL,1471,270


In [ ]:
full_ranks = pd.DataFrame()  #Condensed dictionary into one df containing team ID, season, and average rank
for i in range(2004, 2023):
  x = teamstats_db[i]
  x['Season'] = i
  full_ranks = pd.concat([full_ranks, x])
full_ranks

,TeamID,rank,Season
0,1112.0,35.257143,2004
1,1130.0,31.647059,2004
2,1153.0,12.416667,2004
3,1163.0,7.500000,2004
4,1181.0,2.138889,2004
...,...,...,...
353,1468.0,195.277778,2022
354,1469.0,277.037037,2022
355,1470.0,208.537037,2022
356,1471.0,273.759259,2022


In [ ]:
seasons_dict = {} #Created dictionary with each key containing the games for the corresponding season
for i in range(2003, 2023):
  games = games_l.loc[games_l['Season'] == i]
  seasons_dict[i] = games
seasons_dict

{2003:       Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  WFGM  \
 0       2003      10     1104      68     1328      62    N      0    27   
 1       2003      10     1272      70     1393      63    N      0    26   
 2       2003      11     1266      73     1437      61    N      0    24   
 3       2003      11     1296      56     1457      50    N      0    18   
 4       2003      11     1400      77     1208      71    N      0    30   
 ...      ...     ...      ...     ...      ...     ...  ...    ...   ...   
 4611    2003     131     1436      56     1131      55    A      0    21   
 4612    2003     132     1181      84     1301      77    N      0    24   
 4613    2003     132     1228      72     1326      59    N      0    26   
 4614    2003     132     1246      64     1280      57    N      0    22   
 4615    2003     132     1328      49     1281      47    N      0    18   
 
       WFGA  ...  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk

In [ ]:
avgs = pd.DataFrame()
#Below I sort stats between the winning and losing teams
#Variables included contained in the index line
w = ['WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR','WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LOR', 'LTO'] 
l = ['LScore', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 'WScore', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WOR', 'WTO']
index = ['Score','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF', 'OScore', 'FGMA', 'FGAA', '3PMA', '3PAA', 'ORA', 'TOA'] 

#Below I create a df containing average box score stats for each team.
#Each season has its own entry.

for year in range(2003, 2023):
  season = seasons_dict[year]
  ids = season['LTeamID']
  ids.append(season['WTeamID'])
  ids = pd.Series(ids.values).sort_values().unique()
  place = pd.DataFrame()
  for id in ids:
    team_wins = season.loc[season['WTeamID'] == id][w]
    team_wins.columns = index
    team_losses = season.loc[season['LTeamID'] == id][l]
    team_losses.columns = index
    box = pd.concat([team_wins, team_losses]).mean()
    box['Season'] = year
    box['TeamID'] = id
    place.loc[id, box.index] = box.values
  avgs = pd.concat([avgs, place])

In [ ]:
avgs

,Score,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,...,PF,OScore,FGMA,FGAA,3PMA,3PAA,ORA,TOA,Season,TeamID
1102,57.250000,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,13.000000,...,18.750000,57.000000,19.285714,42.428571,4.750000,12.428571,9.607143,12.964286,2003.0,1102.0
1103,78.777778,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,15.222222,...,19.851852,78.148148,27.777778,57.000000,6.666667,18.370370,12.037037,15.333333,2003.0,1103.0
1104,69.285714,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,12.107143,...,18.035714,65.000000,23.250000,55.500000,6.357143,19.142857,10.892857,13.857143,2003.0,1104.0
1105,71.769231,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,14.538462,...,20.230769,76.653846,27.000000,58.961538,6.269231,17.538462,13.192308,18.807692,2003.0,1105.0
1106,63.607143,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,11.678571,...,18.178571,63.750000,21.714286,53.392857,4.785714,15.214286,11.321429,15.071429,2003.0,1106.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,66.600000,25.050000,54.850000,7.150000,22.100000,9.350000,12.550000,5.900000,20.650000,13.300000,...,16.050000,70.900000,25.300000,55.250000,9.150000,26.350000,8.900000,11.250000,2022.0,1468.0
1469,69.526316,24.105263,58.473684,6.473684,21.631579,14.842105,21.473684,8.736842,23.631579,14.631579,...,19.842105,77.526316,28.315789,60.789474,7.789474,22.789474,8.894737,12.473684,2022.0,1469.0
1470,63.428571,22.619048,54.904762,5.142857,16.666667,13.047619,17.571429,7.380952,19.095238,10.142857,...,18.380952,66.476190,23.142857,51.190476,7.571429,22.523810,8.666667,15.809524,2022.0,1470.0
1471,67.100000,22.700000,52.400000,8.100000,23.700000,13.600000,18.600000,4.800000,21.950000,11.950000,...,15.050000,71.100000,28.150000,59.650000,5.900000,19.050000,8.500000,11.900000,2022.0,1471.0


In [ ]:
k = avgs.loc[avgs['Season'] != 2003]
total_ref = k.merge(full_ranks, on=['Season', 'TeamID'])
total_ref = total_ref.set_index(['Season', 'TeamID'])
total_ref['Ast/TO'] = total_ref['Ast'] / total_ref['TO']
team_avgs = total_ref.copy()

In [ ]:
#Created new variables built off the ones I have to stress the importance of these stats
#This was based off my knowledge of basketball, since I know point per possession 
#efficiency metrics, 3pt shooting, and FT shooting are very important in march

for year in range(2004, 2023): #calculating team stats to be used in analysis later
  team_avgs.loc[:,['possessions']] = team_avgs['FGA'] - team_avgs['OR'] + team_avgs['TO'] + .44 * team_avgs['FTA'] #We can estimate possessions using this formula
  team_avgs.loc[:,['o_rtg']] = team_avgs['Score'] / team_avgs['possessions']
  team_avgs.loc[:,['d_rtg']] = team_avgs['OScore'] / team_avgs['possessions']
  team_avgs.loc[:,['3P%']] = team_avgs['FGM3'] / team_avgs['FGA3']
  team_avgs.loc[:,['FT%']] = team_avgs['FTM'] / team_avgs['FTA']
team_avgs

Score        FGM        FGA       FGM3       FGA3  \
Season TeamID                                                          
2004.0 1102.0  60.178571  20.285714  42.035714   8.464286  22.214286   
       1103.0  71.750000  24.857143  53.285714   6.178571  17.035714   
       1104.0  72.206897  24.896552  55.000000   7.137931  18.620690   
       1105.0  69.793103  23.620690  57.896552   6.206897  19.551724   
       1106.0  61.433333  20.566667  48.733333   5.533333  16.566667   
...                  ...        ...        ...        ...        ...   
2022.0 1468.0  66.600000  25.050000  54.850000   7.150000  22.100000   
       1469.0  69.526316  24.105263  58.473684   6.473684  21.631579   
       1470.0  63.428571  22.619048  54.904762   5.142857  16.666667   
       1471.0  67.100000  22.700000  52.400000   8.100000  23.700000   
       1472.0  72.736842  26.105263  60.000000  10.736842  30.052632   

                     FTM        FTA         OR         DR        Ast  ...  \
Season TeamID                                                         ...   
2004.0 1102.0  11.142857  15.714286   6.142857  15.357143  13.250000  ...   
       1103.0  15.857143  24.571429  11.214286  22.357143  15.107143  ...   
       1104.0  15.275862  21.862069  11.310345  24.000000  11.862069  ...   
       1105.0  16.344828  23.724138  12.517241  23.965517  12.344828  ...   
       1106.0  14.766667  22.466667  11.700000  22.766667  10.266667  ...   
...                  ...        ...        ...        ...        ...  ...   
2022.0 1468.0   9.350000  12.550000   5.900000  20.650000  13.300000  ...   
       1469.0  14.842105  21.473684   8.736842  23.631579  14.631579  ...   
       1470.0  13.047619  17.571429   7.380952  19.095238  10.142857  ...   
       1471.0  13.600000  18.600000   4.800000  21.950000  11.950000  ...   
       1472.0   9.789474  12.631579   5.736842  19.368421  11.842105  ...   

                    3PAA        ORA        TOA        rank    Ast/TO  \
Season TeamID                                                          
2004.0 1102.0  11.071429   9.285714  16.428571   45.212121  1.228477   
       1103.0  18.964286  13.571429  13.535714  174.030303  1.034230   
       1104.0  20.862069  11.517241  13.448276   35.090909  0.893506   
       1105.0  18.517241  12.931034  16.068966  287.937500  0.763326   
       1106.0  16.300000  11.333333  15.333333  251.031250  0.596899   
...                  ...        ...        ...         ...       ...   
2022.0 1468.0  26.350000   8.900000  11.250000  195.277778  1.364103   
       1469.0  22.789474   8.894737  12.473684  277.037037  0.972028   
       1470.0  22.523810   8.666667  15.809524  208.537037  0.995327   
       1471.0  19.050000   8.500000  11.900000  273.759259  0.901887   
       1472.0  23.789474   7.526316  10.631579  295.962264  1.490066   

               possessions     o_rtg     d_rtg       3P%       FT%  
Season TeamID                                                       
2004.0 1102.0    53.592857  1.122884  0.940957  0.381029  0.709091  
       1103.0    67.490000  1.063120  1.068941  0.362683  0.645349  
       1104.0    66.584828  1.084435  1.012968  0.383333  0.698738  
       1105.0    71.990345  0.969479  1.019294  0.317460  0.688953  
       1106.0    64.118667  0.958119  0.945123  0.334004  0.657270  
...                    ...       ...       ...       ...       ...  
2022.0 1468.0    64.222000  1.037028  1.103983  0.323529  0.745020  
       1469.0    74.237895  0.936534  1.044296  0.299270  0.691176  
       1470.0    65.445714  0.969178  1.015746  0.308571  0.742547  
       1471.0    69.034000  0.971985  1.029927  0.341772  0.731183  
       1472.0    67.768421  1.073315  1.152532  0.357268  0.775000  

[6208 rows x 28 columns]

In [ ]:
reg_games = games_s.loc[games_s['Season'] > 2003]
reg_games #pulled condensed team game logs for all seasons

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
78664,2004,10,1266,52,1385,45,A,0
78665,2004,10,1448,85,1272,76,N,0
78666,2004,11,1338,71,1104,62,N,0
78667,2004,11,1386,73,1211,66,N,0
78668,2004,12,1227,77,1414,65,N,0
...,...,...,...,...,...,...,...,...
174466,2022,98,1400,79,1242,76,H,0
174467,2022,98,1411,66,1126,63,A,0
174468,2022,98,1422,68,1441,49,A,0
174469,2022,98,1438,69,1181,68,A,0


In [ ]:
homes = []  #create arrays containing home and away IDs for each game
aways = []
home_win = []  #array containing whether the home team won or not
years = [] #array contains season game occurred in
#These arrays share indexing
for i in range(0, len(reg_games)):
  x = reg_games.iloc[i]
  if x['WLoc'] == 'A':
    aways.append(x['WTeamID'])
    homes.append(x['LTeamID'])
    home_win.append(0)
    aways.append(x['LTeamID'])
    homes.append(x['WTeamID'])
    home_win.append(1)
    years.append(x['Season'])
    years.append(x['Season'])
  else:
    homes.append(x['WTeamID'])
    aways.append(x['LTeamID'])
    home_win.append(1)
    homes.append(x['LTeamID'])
    aways.append(x['WTeamID'])
    home_win.append(0)
    years.append(x['Season'])
    years.append(x['Season'])

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

normal = Normalizer() 
#I am choosing to normalize all the data because I am using a K-nearest neighbors algorithm
#Also do not want to make any assumptions about the data, though sample size is large

In [ ]:
cols = ['rank', 'o_rtg', 'd_rtg', '3P%', 'FT%', 'Ast/TO', 'TOA']
stats = team_avgs[cols]
transformed = normal.fit_transform(stats)

transformed[0]
new_trans = pd.DataFrame()
new_trans['Season'] = total_ref.reset_index()['Season']
new_trans['TeamID'] = total_ref.reset_index()['TeamID']
new_trans.loc[:,cols] = transformed
new_trans = new_trans.set_index(['Season', 'TeamID'])
new_trans

#This database contains the normalized stats I will use for each team in each year

rank     o_rtg     d_rtg       3P%       FT%    Ast/TO  \
Season TeamID                                                               
2004.0 1102.0  0.939002  0.023321  0.019543  0.007914  0.014727  0.025514   
       1103.0  0.996925  0.006090  0.006123  0.002078  0.003697  0.005925   
       1104.0  0.932575  0.028820  0.026921  0.010187  0.018570  0.023746   
       1105.0  0.998428  0.003362  0.003534  0.001101  0.002389  0.002647   
       1106.0  0.998118  0.003810  0.003758  0.001328  0.002613  0.002373   
...                 ...       ...       ...       ...       ...       ...   
2022.0 1468.0  0.998282  0.005301  0.005644  0.001654  0.003809  0.006973   
       1469.0  0.998965  0.003377  0.003766  0.001079  0.002492  0.003505   
       1470.0  0.997098  0.004634  0.004857  0.001475  0.003550  0.004759   
       1471.0  0.999033  0.003547  0.003759  0.001247  0.002668  0.003291   
       1472.0  0.999324  0.003624  0.003892  0.001206  0.002617  0.005031   

                    TOA  
Season TeamID            
2004.0 1102.0  0.341202  
       1103.0  0.077539  
       1104.0  0.357401  
       1105.0  0.055719  
       1106.0  0.060966  
...                 ...  
2022.0 1468.0  0.057511  
       1469.0  0.044979  
       1470.0  0.075592  
       1471.0  0.043427  
       1472.0  0.035898  

[6208 rows x 7 columns]

In [ ]:
nearest = pd.DataFrame()
nearest['home'] = homes #Creates variable containing home team id for each game
nearest['away'] = aways #Creates variable containing away team id for each game
nearest['h_win'] = home_win #Variable for whether home team won (1) or lost (0)
nearest['season'] = years #season each game occured in

for szn in range(2004, 2023):
  missing = []
  for i in range(1101, 1472):
    if i not in new_trans.reset_index().loc[new_trans.reset_index()['Season'] == szn]['TeamID'].tolist():
      missing.append(i) #pulls list of team IDs that do not occur in new_trans
  t = (((nearest['home'].isin(missing)) &  (nearest['season'] == szn)) | ((nearest['away'].isin(missing)) & (nearest['season'] == szn)))
  nearest = nearest.loc[-t]

  #Removes games from the game databse who include a team missing from new_trans

In [ ]:
diff = pd.DataFrame()
#This df takes the difference in normalized stats between the home and away teams
#There are two entries for each game to be able to find k-nearest comparisons for the home and away teams
#For instance, suppose team 1 is really good and team 2 is really bad, and each stat is positively in favor of home team
#So the 7 variables would be something like [1, 1, 1, 1, 1, 1, 1].
#This would not identify a good matchup comparison from the database if the stronger team
#was on the road because it would instead read [-1, -1, -1, -1, -1, -1]
#and the k-nearest algorithm would think it was a really bad matchup comparison
#NOTE: this method prevented home court advantage from being factored.
#I thought this would be OK because the NCAA Tournament is entirely neutral sites.
for k in range(0, len(nearest)):
  h = nearest.iloc[k]['home']
  a = nearest.iloc[k]['away']
  diff.loc[k, new_trans.columns] = new_trans.loc[(nearest.iloc[k]['season'],h),:] - new_trans.loc[(nearest.iloc[k]['season'],a),:]
diff

,rank,o_rtg,d_rtg,3P%,FT%,Ast/TO,TOA
0,0.009338,-0.007353,-0.006395,-0.002940,-0.005233,-0.006389,-0.082195
1,-0.009338,0.007353,0.006395,0.002940,0.005233,0.006389,0.082195
2,-0.210670,0.014272,0.013092,0.004789,0.009198,0.009337,0.266599
3,0.210670,-0.014272,-0.013092,-0.004789,-0.009198,-0.009337,-0.266599
4,-0.442327,0.046125,0.034117,0.011002,0.025371,0.057834,0.503574
...,...,...,...,...,...,...,...
180747,-0.001682,0.002155,0.001839,0.000885,0.001745,0.001606,0.031960
180748,-0.422035,0.066189,0.051883,0.021049,0.040889,0.095470,0.677323
180749,0.422035,-0.066189,-0.051883,-0.021049,-0.040889,-0.095470,-0.677323
180750,-0.014873,0.011529,0.008960,0.004608,0.007284,0.015473,0.123270


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

sample = pd.read_csv('gdrive/My Drive/march_madness/MSampleSubmissionStage2.csv')
#The above csv contained each possible matchup that could occur in march madness based on bracket

X_train = diff #Matchup difference game logs
y_train = nearest['h_win'] #Whether home team won

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

X_train_ = X_train.iloc[:159999]  #Tested model
y_train_ = nearest.iloc[:159999]['h_win']
X_test = X_train.iloc[160000:]
y_test = nearest.iloc[160000:]['h_win']


knn = KNeighborsClassifier()
grid_search = GridSearchCV(
    knn,
    param_grid={"n_neighbors": [181, 186, 188, 191, 194]},
    scoring="neg_log_loss",
    cv=6
)

grid_search.fit(X_train, y_train)
grid_search.best_params_

#Ran an error analysis and k=181 returned the smallest logloss
#Note that I tested several ranges of numbers and it eventually repeatedly chose 181 over all tested numbers

In [ ]:
model = KNeighborsClassifier(n_neighbors = 181)
model.fit(X_train, y_train) #Fit k=181 on matchup differences to predict whether home team wins

In [ ]:
def generate_matches(sample): #Generates team difference stats for each possible matchup

  kaggle = pd.DataFrame()

  for k in range(0, len(sample)):
    yr = int(sample.iloc[k]['ID'][0:4])
    team1 = int(sample.iloc[k]['ID'][5:9])
    team2 = int(sample.iloc[k]['ID'][10:14])
    kaggle.loc[k, new_trans.columns] = new_trans.loc[(yr,team1),:] - new_trans.loc[(yr,team2),:]
      
  return kaggle

matches = generate_matches(sample)

In [ ]:
def pred_winner(matches): #Predicts win probability for each possible matchup
    probs = []
    probs = model.predict_proba(matches)
    win_prob = []
    for i in range(0, len(probs)):
      win_prob.append(probs[i][1])
    return win_prob
results = pred_winner(matches)
submission1 = pd.DataFrame()
submission1['ID'] = sample['ID']
submission1['Pred'] = results

submission1 = submission1.set_index('ID')
submission1.to_csv('/content/gdrive/My Drive/march_madness/cleaned_data/SubmissionStage2.csv')

In [ ]:
submission1 #Submission for kaggle competition

,Pred
ID,
2022_1103_1104,0.039604
2022_1103_1112,0.000000
2022_1103_1116,0.079208
2022_1103_1120,0.009901
2022_1103_1124,0.000000
...,...
2022_1458_1461,0.722772
2022_1458_1463,0.920792
2022_1460_1461,0.089109


**Post-tourney analysis**

My submission scored very poorly because I failed to consider the ramifications of my model identifying a matchup as 100% win probability. This happened in the first round matchup between St. Peters and Kentucky, with Kentucky predicted to win 100% of matchups. I researched solutions for this and arrived at the possibility of "clipping" the probabilities at a number (such as 95% or something slightly more forgiveable to logloss) to prevent this from happening. Many people in the competition did this, and I wanted to test different methods of clipping so I played around with it below. I ran logloss calculations on my clipped results and believe I would have been around top 50 had I clipped win %s at 95%.

In [ ]:
from sklearn.model_selection import cross_val_score

model = KNeighborsClassifier(n_neighbors = 181)
model.fit(X_train, y_train)

scores = cross_val_score(model, 
                         X=X_train,
                         y=-1 * (y_train-1),
                         scoring="neg_log_loss",
                         cv=10)
np.sqrt(-scores).mean()

0.7045899331213249

In [ ]:
subs = pd.read_csv('/content/gdrive/My Drive/march_madness/cleaned_data/SubmissionStage2.csv')

edit = subs.copy()
for i in range(0, len(edit)):
  prob = edit.iloc[i]['Pred']
  if prob < .35:
    edit.iloc[i, 1] = prob +.07
  if prob > .65:
    edit.iloc[i, 1] = prob - .07
  if prob >= .35 and prob <= .45:
    edit.iloc[i, 1] = prob +.03
  if prob <= .65 and prob >= .55:
    edit.iloc[i, 1] = prob -.03


In [ ]:
latesubmission = edit.set_index('ID')
latesubmission.to_csv('/content/gdrive/My Drive/march_madness/cleaned_data/lateSubmission.csv')